# Data Cleaning

In [17]:
import pandas as pd

cashrequestdf = pd.read_csv("project_dataset/cashreq.csv")
feesdf = pd.read_csv("project_dataset/fees.csv")

In [18]:
# updated column names as they have the same column name. Could create problems when merged together.
cashrequestdf = cashrequestdf.rename(columns={"id": "cash_request_id", "status": "cash_request_status", "created_at": "cash_request_created_at", "updated_at": "cash_request_updated_at", "reason": "cash_request_reason"})
feesdf = feesdf.rename(columns={"id": "fee_id", "status": "fee_status", "created_at": "fee_created_at", "updated_at": "fee_updated_at", "reason": "fee_reason"})

In [19]:
# Add boolean flags
cashrequestdf["was_moderated"] = cashrequestdf["moderated_at"].notna()
cashrequestdf["was_deleted_account"] = cashrequestdf["deleted_account_id"].notna()
cashrequestdf["was_sent"] = cashrequestdf["send_at"].notna()
cashrequestdf["was_money_back"] = cashrequestdf["money_back_date"].notna()

In [20]:
# replace user id as string
cashrequestdf["user_id"] = cashrequestdf["user_id"].astype("Int64")
cashrequestdf.sample(5)

,cash_request_id,amount,cash_request_status,cash_request_created_at,cash_request_updated_at,user_id,moderated_at,deleted_account_id,reimbursement_date,cash_request_received_date,money_back_date,transfer_type,send_at,recovery_status,reco_creation,reco_last_update,was_moderated,was_deleted_account,was_sent,was_money_back
16114,11146,100.0,money_back,2020-08-06 13:00:49.182477+00,2020-12-18 13:10:46.478407+00,2011,2020-08-06 14:06:38.467175+00,NaN,2020-09-03 22:00:00+00,2020-08-14,2020-09-07 22:00:00+00,regular,2020-08-13 13:00:49.182054+00,NaN,NaN,NaN,True,False,True,True
2859,5960,100.0,rejected,2020-06-30 12:57:02.374814+00,2020-06-30 16:41:40.242354+00,14098,2020-06-30 16:41:40.238045+00,NaN,2020-07-28 22:00:00+00,NaN,NaN,regular,NaN,NaN,NaN,NaN,True,False,False,False
3285,11219,50.0,rejected,2020-08-06 20:15:50.356838+00,2020-08-07 09:51:34.244977+00,<NA>,2020-08-07 09:51:34.24248+00,13808.0,2020-08-21 23:51:00+00,NaN,NaN,instant,2020-08-13 20:15:50.356237+00,NaN,NaN,NaN,True,True,True,False
7724,1075,100.0,money_back,2020-04-08 06:33:50.522182+00,2020-11-04 12:55:25.635499+00,5267,2020-04-08 09:37:37.431854+00,NaN,2020-05-03 22:00:00+00,2020-04-09,2020-05-12 20:20:42.331435+00,regular,NaN,NaN,NaN,NaN,True,False,False,True
19269,20984,100.0,money_back,2020-10-14 07:23:26.65207+00,2020-12-18 13:12:01.624307+00,5286,NaN,NaN,2020-10-29 07:23:14.232+00,2020-10-15,2020-10-30 20:34:54.208108+00,instant,2020-10-14 07:25:42.422046+00,NaN,NaN,NaN,False,False,True,True


In [21]:
# Check for invalid time sequences
cashrequestdf["invalid_date_sequence"] = ((cashrequestdf["cash_request_updated_at"] < cashrequestdf["cash_request_created_at"]) | (cashrequestdf["money_back_date"] < cashrequestdf["cash_request_created_at"]))

In [22]:
print(cashrequestdf.shape)
cashrequestdf.isnull().sum()

(23970, 21)


cash_request_id                   0
amount                            0
cash_request_status               0
cash_request_created_at           0
cash_request_updated_at           0
user_id                        2103
moderated_at                   7935
deleted_account_id            21866
reimbursement_date                0
cash_request_received_date     7681
money_back_date                7427
transfer_type                     0
send_at                        7329
recovery_status               20640
reco_creation                 20640
reco_last_update              20640
was_moderated                     0
was_deleted_account               0
was_sent                          0
was_money_back                    0
invalid_date_sequence             0
dtype: int64

In [23]:
# 1. add add missing user_id values with deleted_account_id
cashrequestdf["user_id"] = cashrequestdf["user_id"].fillna(cashrequestdf["deleted_account_id"])
cashrequestdf.isnull().sum()

cash_request_id                   0
amount                            0
cash_request_status               0
cash_request_created_at           0
cash_request_updated_at           0
user_id                           0
moderated_at                   7935
deleted_account_id            21866
reimbursement_date                0
cash_request_received_date     7681
money_back_date                7427
transfer_type                     0
send_at                        7329
recovery_status               20640
reco_creation                 20640
reco_last_update              20640
was_moderated                     0
was_deleted_account               0
was_sent                          0
was_money_back                    0
invalid_date_sequence             0
dtype: int64

In [24]:
# 2. add cohort depending on user first cash request

# 2.1 get df with only date of first cash request per user id

only_userid_and_date = cashrequestdf[["user_id", "cash_request_created_at"]] # New DF only with user ids and cash request created date
print("unique users", only_userid_and_date["user_id"].nunique())
only_userid_and_date = only_userid_and_date.sort_values(by="cash_request_created_at", ascending=True) # order by date DESC
only_userid_and_date = only_userid_and_date.drop_duplicates(subset=["user_id"], keep="first") # removing duplicates user ids to only get the date of first cash request.
only_userid_and_date = only_userid_and_date.rename(columns={"cash_request_created_at": "user_first_cash_request_date"}) # changing name of column for legibility
print("rows users, should match above print", only_userid_and_date.shape[0])

unique users 11793
rows users, should match above print 11793


In [25]:
# 2.2 assign cohort name depending on "first_cash_request_date"
from datetime import datetime

only_userid_and_date["user_first_cash_request_date"] = pd.to_datetime(only_userid_and_date["user_first_cash_request_date"])  # Convert column to datetime

def assign_cohort(each_row_value):
  if not each_row_value:
    print(each_row_value)
  year = each_row_value.year
  month = each_row_value.strftime("%m") # adds a 0 in front if only one digit
  return f"{year}-{month}"

only_userid_and_date["cohort"] = only_userid_and_date["user_first_cash_request_date"].apply(assign_cohort) # adds new column with cohort name

# 2.3 merged new "cohort" column with original
cashrequestdf = pd.merge(left=cashrequestdf, right=only_userid_and_date, left_on="user_id", right_on="user_id", how="left") # left just to make sure we don't remove any cash requests
cashrequestdf.shape
# TODO used for first metric: 1. ***Frequency of Service Usage:**

(23970, 23)

In [26]:
# 2.4 create clean data csv
cashrequestdf.to_csv("project_dataset/cashreq-clean.csv")

In [27]:
# 3.2. unir los dataframes  (incident rates esta en la tabla de fees)
fees_with_user_id_and_cohort = pd.merge(left=feesdf, right=cashrequestdf, left_on="cash_request_id", right_on="cash_request_id", how="left")
print(feesdf.shape[0])
print(fees_with_user_id_and_cohort.shape[0]) # same as above
# TODO used for second metric: 2. **Incident Rate:**
# TODO used for third metric: 3. **Revenue Generated by the Cohort:**


21061
21061


In [28]:
# 3.2 create clean joined fees/cashreq data to csv
fees_with_user_id_and_cohort.to_csv("project_dataset/fees-with-cohort-and-user-id-clean.csv")

# Next Steps

- Analize MVP metrics
  1. Frequency of Service Usage
  2. Incident Rate
  3. Revenue Generated by the Cohort